##Selenium



In [ ]:
!apt-get update &> /dev/null
!pip install selenium &> /dev/null
!apt-get install chromium chromium-driver &> /dev/null

##Imports, Drive connection & constants setup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import numpy as np
from google.colab import drive
import pandas as pd
import warnings

drive.mount('/content/drive')
path = '/content/drive/MyDrive/STOCK PREDICT/BCE/'
start_date = '28/06/2010'
end_date = '28/07/2023'
update = True
timeout = 3 # waiting period (in seconds) for loading
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
if update:
  start_date = pd.read_csv(path + 'data.csv').iloc[-1].date.replace("-", "/").split('/')

  # start_date = '/'.join([start_date[1], start_date[0], start_date[2]])
  start_date = '/'.join([start_date[2], start_date[1], start_date[0]])

  end_date = ''
  # end_date = '12/10/2023'

print('Update from after:', start_date, 'to', end_date)

Update from after: 12/10/2023 to 


##Web driver

In [ ]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--vebose')
  options.add_argument('--no-sandbox')
  options.add_argument('--headless')
  options.add_argument('--disable-gpu')
  options.add_argument('--diable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)
  return driver

driver = web_driver()
driver2 = web_driver() # for back-up data source
driver3 = web_driver() # for back-up data source

#WEB SCRAPING

##Helper functions

In [ ]:
# Get prices of the stock & other related data
def get_prices(start_date='', end_date=''):
  driver.get('https://s.cafef.vn/lich-su-giao-dich-bce-1.chn')

  date = driver.find_element(by="xpath", value='//input[contains(@id, "date-inp-disclosure")]')
  search = driver.find_element(by="xpath", value='//div[contains(@id, "owner-find")]')

  date.send_keys(start_date + " - " + end_date)
  search.click()

  time.sleep(timeout)

  rows = []
  # page = 1

  while True:
    containers = driver.find_elements(by="xpath", value='//tr[contains(@class, "Owner")]')
    for c in containers:
      row_data = c.text.split()
      datestr = row_data[0].split('/')
      date = np.datetime64('-'.join([datestr[2], datestr[1], datestr[0]]) + 'T00:00:00.000000000')
      close = float(row_data[1].replace(',',''))*1000
      volume1 = float((row_data[5].replace(',','')))
      value1 = float((row_data[6]).replace(',',''))
      volume2 = float((row_data[7].replace(',','')))
      value2 = float((row_data[8]).replace(',',''))
      open = float(row_data[9].replace(',',''))*1000
      high = float(row_data[10].replace(',',''))*1000
      low = float(row_data[11].replace(',',''))*1000
      rows.append([date, open, close, high, low, volume1, value1, volume2, value2])
    # print('Page: ___ : ',page)
    try:
      next = driver.find_element(by="xpath", value='//a[contains(@title, "Next to")]')
      next.click()
      # page += 1
    except:
      break
    time.sleep(timeout)

  return rows

In [ ]:
# Get orders from another page if main page does not have
def get_orders_backup(date='', ref_dates=[]):
  driver2.get('https://www.stockbiz.vn/Stocks/bce/TradingStatistics.aspx')

  date1 = driver2.find_element(by="xpath", value='//*[@id="ctl00_webPartManager_wp425243205_wp378545232_dtStartDate_picker_picker"]')
  date2 = driver2.find_element(by="xpath", value='//*[@id="ctl00_webPartManager_wp425243205_wp378545232_dtEndDate_picker_picker"]')
  search = driver2.find_element(by="xpath", value='//*[@id="ctl00_webPartManager_wp425243205_wp378545232_btnView"]')

  ActionChains(driver2).click().perform()
  date1.send_keys(date)
  date2.send_keys(date)
  search.click()

  time.sleep(timeout)

  container = driver2.find_element(by="xpath", value='//*[@id="ctl00_webPartManager_wp425243205_wp378545232_cbTradingResult"]/table/tbody/tr[2]')
  row_data = container.text.split()
  buy_order = float((row_data[2].replace('.','')))
  buy_vol = float((row_data[3].replace('.','')))
  sell_order = float((row_data[4].replace('.','')))
  sell_vol = float((row_data[5].replace('.','')))

  if (buy_order * buy_vol * sell_order * sell_vol == 0) or (date not in ref_dates):
    buy_order, buy_vol, sell_order, sell_vol = get_orders_backup(date)

  return [buy_order, buy_vol, sell_order, sell_vol]


def get_orders_backup_2(date=''):
  driver3.get('https://finance.vietstock.vn/ket-qua-giao-dich?tab=thong-ke-lenh&exchange=1&code=502')

  date1 = driver3.find_element(by="xpath", value='//*[@id="txtFromDate"]/input')
  date2 = driver3.find_element(by="xpath", value='//*[@id="txtToDate"]/input')
  search = driver3.find_element(by="xpath", value='//*[@id="trading-result"]/div/div[1]/div[1]/div/div[2]/button')

  date1.clear()
  date2.clear()

  ActionChains(driver2).click().perform()
  date1.send_keys(date)
  date2.send_keys(date)
  search.click()

  time.sleep(timeout)

  container = driver3.find_element(by="xpath", value='//*[@id="statistic-order"]/table/tbody/tr')
  row_data = container.text.split()
  buy_order = float((row_data[10].replace(',','')))
  buy_vol = float((row_data[13].replace(',','')))
  sell_order = float((row_data[11].replace(',','')))
  sell_vol = float((row_data[14].replace(',','')))

  return [buy_order, buy_vol, sell_order, sell_vol]


# Get order info from main page
def get_orders(start_date='', end_date='', ref_dates=[]):
  driver.get('https://s.cafef.vn/lich-su-giao-dich-bce-2.chn')

  date = driver.find_element(by="xpath", value='//input[contains(@id, "date-inp-disclosure")]')
  search = driver.find_element(by="xpath", value='//div[contains(@id, "owner-find")]')

  date.send_keys(start_date + " - " + end_date)
  search.click()

  time.sleep(timeout)

  rows = []
  # page = 1

  while True:
    containers = driver.find_elements(by="xpath", value='//tr[contains(@class, "Owner")]')
    for c in containers:
      row_data = c.text.split()
      datestr = row_data[0].split('/')
      date = np.datetime64('-'.join([datestr[2], datestr[1], datestr[0]]) + 'T00:00:00.000000000')
      buy_order = float((row_data[5].replace(',','')))
      buy_vol = float((row_data[6].replace(',','')))
      sell_order = float((row_data[8].replace(',','')))
      sell_vol = float((row_data[9].replace(',','')))

      if (buy_order * buy_vol * sell_order * sell_vol == 0) or (date not in ref_dates):
        buy_order, buy_vol, sell_order, sell_vol = get_orders_backup(row_data[0], ref_dates)

      rows.append([date, buy_order, buy_vol, sell_order, sell_vol])
    # print('Page: ___ : ',page)
    try:
      next = driver.find_element(by="xpath", value='//a[contains(@title, "Next to")]')
      next.click()
      # page += 1
    except:

      break
    time.sleep(timeout)

  return rows

In [ ]:
# Get prices of the market & other related data
def get_market_prices(start_date='', end_date=''):
  driver.get('https://s.cafef.vn/lich-su-giao-dich-vnindex-1.chn#data')

  date = driver.find_element(by="xpath", value='//input[contains(@id, "date-inp-disclosure")]')
  search = driver.find_element(by="xpath", value='//div[contains(@id, "owner-find")]')

  date.send_keys(start_date + " - " + end_date)
  search.click()

  time.sleep(timeout)

  rows = []
  # page = 1

  while True:
    containers = driver.find_elements(by="xpath", value='//tr[contains(@class, "Owner")]')
    for c in containers:
      row_data = c.text.split()
      datestr = row_data[0].split('/')
      date = np.datetime64('-'.join([datestr[2], datestr[1], datestr[0]]) + 'T00:00:00.000000000')
      close = float(row_data[1].replace(',',''))
      volume1 = float((row_data[5].replace(',','')))
      value1 = float((row_data[6].replace(',','')))
      volume2 = float((row_data[7].replace(',','')))
      value2 = float((row_data[8].replace(',','')))
      open = float(row_data[9].replace(',',''))
      high = float(row_data[10].replace(',',''))
      low = float(row_data[11].replace(',',''))
      rows.append([date, open, close, high, low, volume1, value1, volume2, value2])
    # print('Page: ___ : ',page)
    try:
      next = driver.find_element(by="xpath", value='//a[contains(@title, "Next to")]')
      next.click()
      # page += 1
    except:
      break
    time.sleep(timeout)

  return rows

## Scrape data

In [ ]:
# Stock prices
df_prices = pd.DataFrame(get_prices(start_date, end_date), columns=['date', 'open', 'close', 'high', 'low', 'volume1', 'value1', 'volume2', 'value2'])
display(df_prices)

,date,open,close,high,low,volume1,value1,volume2,value2
0,2023-10-13,5800.0,5780.0,5800.0,5490.0,113600.0,0.64,0.0,0.0
1,2023-10-12,5950.0,5900.0,6000.0,5850.0,28700.0,0.17,0.0,0.0


In [ ]:
# Stock orders
df_orders = pd.DataFrame(get_orders(start_date, end_date, df_prices['date'].values), columns=['date', 'buy_order', 'buy_vol', 'sell_order', 'sell_vol'])
display(df_orders)

,date,buy_order,buy_vol,sell_order,sell_vol
0,2023-10-13,175.0,244900.0,62.0,261000.0
1,2023-10-12,80.0,83500.0,74.0,59600.0


In [ ]:
# Market prices
df_market_prices = pd.DataFrame(get_market_prices(start_date, end_date), columns=['date', 'open_HSX', 'close_HSX', 'high_HSX', 'low_HSX', 'volume1_HSX', 'value1_HSX', 'volume2_HSX', 'value2_HSX'])
display(df_market_prices)

,date,open_HSX,close_HSX,high_HSX,low_HSX,volume1_HSX,value1_HSX,volume2_HSX,value2_HSX
0,2023-10-13,1151.61,1154.06,1154.93,1138.99,581997586.0,12834.79,0.0,0.0
1,2023-10-12,1150.81,1152.34,1156.73,1148.13,613297500.0,13683.57,0.0,0.0


In [ ]:
# Join all data into one dataframe
df_joined = pd.merge(df_prices, df_orders, on='date', how='left')
df_joined = pd.merge(df_joined, df_market_prices, on='date', how='left')
df_joined.sort_values(by='date', inplace=True)
df_joined.reset_index(drop=True, inplace=True)

df_joined.to_csv(path+'web_scrape.csv', index=False)
display(df_joined)

,date,open,close,high,low,volume1,value1,volume2,value2,buy_order,...,sell_order,sell_vol,open_HSX,close_HSX,high_HSX,low_HSX,volume1_HSX,value1_HSX,volume2_HSX,value2_HSX
0,2023-10-12,5950.0,5900.0,6000.0,5850.0,28700.0,0.17,0.0,0.0,80.0,...,74.0,59600.0,1150.81,1152.34,1156.73,1148.13,613297500.0,13683.57,0.0,0.0
1,2023-10-13,5800.0,5780.0,5800.0,5490.0,113600.0,0.64,0.0,0.0,175.0,...,62.0,261000.0,1151.61,1154.06,1154.93,1138.99,581997586.0,12834.79,0.0,0.0


#DATA CLEANING

In [ ]:
df_joined = pd.read_csv(path+'web_scrape.csv')
df_joined

,date,open,close,high,low,volume1,value1,volume2,value2,buy_order,...,sell_order,sell_vol,open_HSX,close_HSX,high_HSX,low_HSX,volume1_HSX,value1_HSX,volume2_HSX,value2_HSX
0,2023-10-12,5950.0,5900.0,6000.0,5850.0,28700.0,0.17,0.0,0.0,80.0,...,74.0,59600.0,1150.81,1152.34,1156.73,1148.13,613297500.0,13683.57,0.0,0.0
1,2023-10-13,5800.0,5780.0,5800.0,5490.0,113600.0,0.64,0.0,0.0,175.0,...,62.0,261000.0,1151.61,1154.06,1154.93,1138.99,581997586.0,12834.79,0.0,0.0


In [ ]:
df_joined.loc[df_joined.isna().any(axis=1), df_joined.isna().any(axis=0)]

""


In [ ]:
# missing_rows = df_joined.loc[df_joined.isna().any(axis=1), df_joined.isna().any(axis=0)].index
# missing_rows

In [ ]:
# missing_columns = df_joined.loc[df_joined.isna().any(axis=1), df_joined.isna().any(axis=0)].columns
# missing_columns

In [ ]:
# for i in missing_rows:
#   date = df_joined.loc[i, 'date'].split('-')
#   day = '0' + date[2] if len(date[2]) < 2 else date[2]
#   month = '0' + date[1] if len(date[1]) < 2 else date[1]
#   year = date[0]
#   date = '/'.join([day, month, year])
#   df_joined.loc[i, missing_columns] = get_orders_backup(date)

# df_joined.loc[missing_rows, missing_columns]

In [ ]:
# df_joined[df_joined.isna().any(axis=1)]

In [ ]:
if update:
  pd.concat([pd.read_csv(path+'data.csv').iloc[-201:], df_joined.iloc[1:]], axis=0).reset_index(drop=True).to_csv(path+'web_scrape_cleaned.csv', index=False)
else:
  df_joined.to_csv(path+'web_scrape_cleaned.csv', index=False)

#FEATURE ENGINEERING

##Helper functions

In [ ]:
def get_ratio(dividend, divisor):
  mask1 = (divisor != 0)
  mask2 = (divisor == 0) & (dividend != 0)
  return np.where(mask1, dividend / divisor,
                  np.where(mask2, dividend.max(), 1))


def get_diff_percent(dividend, divisor):
  mask1 = (divisor != 0)
  mask2 = (divisor == 0) & (dividend != 0)
  return np.where(mask1, dividend / divisor - 1,
                  np.where(mask2, 1, 0))


def cross(x1, x2, dir='up'):
  if dir == 'up':
    return np.where(((x1.shift() < x2.shift()) & (x1 > x2)) | ((x1.shift(2) < x2.shift(2)) & (x1.shift() == x2.shift()) & (x1 > x2)), 1, -1)
  if dir == 'down':
    return np.where(((x1.shift() > x2.shift()) & (x1 < x2)) | ((x1.shift(2) > x2.shift(2)) & (x1.shift() == x2.shift()) & (x1 < x2)), 1, -1)


def compare(x1, x2, dir='greater'):
    if dir == 'greater':
      return np.where(x1 > x2, 1, -1)
    if dir == 'smaller':
      return np.where(x1 < x2, 1, -1)

##Add features

In [ ]:
df_joined = pd.read_csv(path+'web_scrape_cleaned.csv')
df_joined

,date,open,close,high,low,volume1,value1,volume2,value2,buy_order,...,FSI_80,FSI_20,BOU,BOL,close_BOU,close_BOL,close_BOU_%,close_BOL_%,close_BOU_yn,BOL_close_yn
0,12/21/2022,5710.0,5800.0,5850.0,5700.0,41100.0,2.370000e+08,0.0,0.0,103.0,...,-1.0,1.0,6699.624225,5364.375775,-899.624225,435.624225,-0.134280,0.081207,-1.0,-1.0
1,12/22/2022,5800.0,5830.0,5840.0,5700.0,16700.0,9.500000e+07,0.0,0.0,79.0,...,-1.0,1.0,6574.650035,5421.349965,-744.650035,408.650035,-0.113261,0.075378,-1.0,-1.0
2,12/23/2022,5810.0,5840.0,5900.0,5710.0,31800.0,1.840000e+08,0.0,0.0,92.0,...,-1.0,1.0,6470.776110,5459.223890,-630.776110,380.776110,-0.097481,0.069749,-1.0,-1.0
3,12/26/2022,5660.0,5650.0,5980.0,5650.0,57100.0,3.270000e+08,0.0,0.0,89.0,...,-1.0,1.0,6447.060508,5384.939492,-797.060508,265.060508,-0.123632,0.049223,-1.0,-1.0
4,12/27/2022,5500.0,5700.0,5790.0,5500.0,15000.0,8.500000e+07,0.0,0.0,87.0,...,-1.0,1.0,6435.676598,5316.323402,-735.676598,383.676598,-0.114312,0.072170,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2023-10-09,5960.0,5950.0,5960.0,5850.0,5700.0,3.000000e-02,0.0,0.0,63.0,...,-1.0,1.0,6395.860985,5360.139015,-445.860985,589.860985,-0.069711,0.110046,-1.0,-1.0
198,2023-10-10,5950.0,5910.0,6000.0,5900.0,27600.0,1.600000e-01,0.0,0.0,73.0,...,-1.0,1.0,6365.597124,5402.402876,-455.597124,507.597124,-0.071572,0.093958,-1.0,-1.0
199,2023-10-11,5990.0,5900.0,5990.0,5750.0,27000.0,1.600000e-01,0.0,0.0,96.0,...,-1.0,1.0,6343.572095,5434.427905,-443.572095,465.572095,-0.069925,0.085671,-1.0,-1.0
200,2023-10-12,5950.0,5900.0,6000.0,5850.0,28700.0,1.700000e-01,0.0,0.0,80.0,...,-1.0,1.0,6324.769919,5461.230081,-424.769919,438.769919,-0.067160,0.080343,-1.0,-1.0


In [ ]:
# PRICE OPEN & CLOSE / HIGH & LOW
df_joined['diff_open'] = df_joined['open'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_open_%'] = df_joined['open'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['diff_close'] = df_joined['close'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_close_%'] = df_joined['close'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['open_close'] = df_joined['open'] - df_joined['close']
df_joined['open_close_%'] = get_diff_percent(df_joined['open'], df_joined['close'])

df_joined['open_ref'] = df_joined['open'] - df_joined['close'].shift()
df_joined['open_ref_%'] = get_diff_percent(df_joined['open'], df_joined['close'].shift())

df_joined['high_low'] = df_joined['high'] - df_joined['low']
df_joined['high_low_%'] = get_diff_percent(df_joined['high'], df_joined['low'])

df_joined['high_open'] = df_joined['high'] - df_joined['open']
df_joined['high_open_%'] = get_diff_percent(df_joined['high'], df_joined['open'])

df_joined['high_close'] = df_joined['high'] - df_joined['close']
df_joined['high_close_%'] = get_diff_percent(df_joined['high'], df_joined['close'])

df_joined['low_open'] = df_joined['low'] - df_joined['open']
df_joined['low_open_%'] = get_diff_percent(df_joined['low'], df_joined['open'])

df_joined['low_close'] = df_joined['low'] - df_joined['close']
df_joined['low_close_%'] = get_diff_percent(df_joined['low'], df_joined['close'])


# VOLUME
df_joined['volume_total'] = df_joined['volume1'] + df_joined['volume2']
df_joined['diff_vol1'] = df_joined['volume1'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_vol1_%'] = df_joined['volume1'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))
df_joined['diff_vol'] = df_joined['volume_total'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_vol_%'] = df_joined['volume_total'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))


# VALUE
df_joined['value_total'] = df_joined['value1'] + df_joined['value2']
df_joined['diff_val1'] = df_joined['value1'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_val1_%'] = df_joined['value1'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))
df_joined['diff_val'] = df_joined['value_total'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_val_%'] = df_joined['value_total'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))


# ORDER BUY & SELL
df_joined['buy_avg_vol'] = df_joined['buy_vol'] / df_joined['buy_order']
df_joined['sell_avg_vol'] = df_joined['sell_vol'] / df_joined['sell_order']

df_joined['buy_sell_vol'] = df_joined['buy_vol'] - df_joined['sell_vol']
df_joined['buy_sell_vol_%'] = get_diff_percent(df_joined['buy_vol'], df_joined['sell_vol'])

df_joined['buy_sell_avg_vol'] = df_joined['buy_avg_vol'] - df_joined['sell_avg_vol']
df_joined['buy_sell_avg_vol_%'] = get_diff_percent(df_joined['buy_avg_vol'], df_joined['sell_avg_vol'])

df_joined['buy_sell_order'] = df_joined['buy_order'] - df_joined['sell_order']
df_joined['buy_sell_order_%'] = get_diff_percent(df_joined['buy_order'], df_joined['sell_order'])

df_joined['buy_order_diff'] = df_joined['buy_order'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['buy_order_diff_%'] = df_joined['buy_order'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['buy_vol_diff'] = df_joined['buy_vol'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['buy_vol_diff_%'] = df_joined['buy_vol'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['buy_avg_vol_diff'] = df_joined['buy_avg_vol'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['buy_avg_vol_diff_%'] = df_joined['buy_avg_vol'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['sell_order_diff'] = df_joined['sell_order'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['sell_order_diff_%'] = df_joined['sell_order'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['sell_vol_diff'] = df_joined['sell_vol'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['sell_vol_diff_%'] = df_joined['sell_vol'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['sell_avg_vol_diff'] = df_joined['sell_avg_vol'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['sell_avg_vol_diff_%'] = df_joined['sell_avg_vol'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))


# HSX OPEN & CLOSE
df_joined['diff_open_HSX'] = df_joined['open_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_open_%_HSX'] = df_joined['open_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['diff_close_HSX'] = df_joined['close_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_close_%_HSX'] = df_joined['close_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))


# HSX VALUE
df_joined['value_total_HSX'] = df_joined['value1_HSX'] + df_joined['value2_HSX']

df_joined['diff_val1_HSX'] = df_joined['value1_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_val1_%_HSX'] = df_joined['value1_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['diff_val2_HSX'] = df_joined['value2_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_val2_%_HSX'] = df_joined['value2_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['diff_val_HSX'] = df_joined['value_total_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_val_%_HSX'] = df_joined['value_total_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))


# HSX VOLUME
df_joined['volume_total_HSX'] = df_joined['volume1_HSX'] + df_joined['volume2_HSX']

df_joined['diff_vol1_HSX'] = df_joined['volume1_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_vol1_%_HSX'] = df_joined['volume1_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['diff_vol2_HSX'] = df_joined['volume2_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_vol2_%_HSX'] = df_joined['volume2_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))

df_joined['diff_vol_HSX'] = df_joined['volume_total_HSX'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
df_joined['diff_vol_%_HSX'] = df_joined['volume_total_HSX'].rolling(window=2).apply(lambda x: get_diff_percent(x.iloc[1], x.iloc[0]))


# SMA
df_joined['SMA_10'] = df_joined['close'].rolling(window=10).mean()
df_joined['SMA_20'] = df_joined['close'].rolling(window=20).mean()
df_joined['SMA_50'] = df_joined['close'].rolling(window=50).mean()
df_joined['SMA_100'] = df_joined['close'].rolling(window=100).mean()
df_joined['SMA_200'] = df_joined['close'].rolling(window=200).mean()

df_joined['close_SMA10'] = get_diff_percent(df_joined['close'], df_joined['SMA_10'])
df_joined['close_SMA50'] = get_diff_percent(df_joined['close'], df_joined['SMA_50'])
df_joined['close_SMA100'] = get_diff_percent(df_joined['close'], df_joined['SMA_100'])
df_joined['close_SMA200'] = get_diff_percent(df_joined['close'], df_joined['SMA_200'])

df_joined['SMA10_SMA20'] = get_diff_percent(df_joined['SMA_10'], df_joined['SMA_20'])
df_joined['SMA10_SMA50'] = get_diff_percent(df_joined['SMA_10'], df_joined['SMA_50'])
df_joined['SMA10_SMA100'] = get_diff_percent(df_joined['SMA_10'], df_joined['SMA_100'])
df_joined['SMA10_SMA200'] = get_diff_percent(df_joined['SMA_10'], df_joined['SMA_200'])

df_joined['close_up_SMA20'] = cross(df_joined['close'], df_joined['SMA_20'], dir='up')
df_joined['close_up_SMA50'] = cross(df_joined['close'], df_joined['SMA_50'], dir='up')
df_joined['close_up_SMA100'] = cross(df_joined['close'], df_joined['SMA_100'], dir='up')
df_joined['close_up_SMA200'] = cross(df_joined['close'], df_joined['SMA_200'], dir='up')

df_joined['close_lo_SMA20'] = cross(df_joined['close'], df_joined['SMA_20'], dir='down')
df_joined['close_lo_SMA50'] = cross(df_joined['close'], df_joined['SMA_50'], dir='down')
df_joined['close_lo_SMA100'] = cross(df_joined['close'], df_joined['SMA_100'], dir='down')
df_joined['close_lo_SMA200'] = cross(df_joined['close'], df_joined['SMA_200'], dir='down')

df_joined['SMA10_up_SMA20'] = cross(df_joined['close'], df_joined['SMA_20'], dir='up')
df_joined['SMA10_up_SMA50'] = cross(df_joined['close'], df_joined['SMA_50'], dir='up')
df_joined['SMA10_up_SMA100'] = cross(df_joined['close'], df_joined['SMA_100'], dir='up')
df_joined['SMA10_up_SMA200'] = cross(df_joined['close'], df_joined['SMA_200'], dir='up')

df_joined['SMA10_lo_SMA20'] = cross(df_joined['close'], df_joined['SMA_20'], dir='down')
df_joined['SMA10_lo_SMA50'] = cross(df_joined['close'], df_joined['SMA_50'], dir='down')
df_joined['SMA10_lo_SMA100'] = cross(df_joined['close'], df_joined['SMA_100'], dir='down')
df_joined['SMA10_lo_SMA200'] = cross(df_joined['close'], df_joined['SMA_200'], dir='down')


# EMA
df_joined['EMA_10'] = df_joined['close'].ewm(span=10, adjust=False).mean()
df_joined['EMA_20'] = df_joined['close'].ewm(span=20, adjust=False).mean()
df_joined['EMA_50'] = df_joined['close'].ewm(span=50, adjust=False).mean()
df_joined['EMA_100'] = df_joined['close'].ewm(span=100, adjust=False).mean()
df_joined['EMA_200'] = df_joined['close'].ewm(span=200, adjust=False).mean()

df_joined['close_EMA10'] = get_diff_percent(df_joined['close'], df_joined['EMA_10'])
df_joined['close_EMA50'] = get_diff_percent(df_joined['close'], df_joined['EMA_50'])
df_joined['close_EMA100'] = get_diff_percent(df_joined['close'], df_joined['EMA_100'])
df_joined['close_EMA200'] = get_diff_percent(df_joined['close'], df_joined['EMA_200'])

df_joined['EMA10_EMA20'] = get_diff_percent(df_joined['EMA_10'], df_joined['EMA_20'])
df_joined['EMA10_EMA50'] = get_diff_percent(df_joined['EMA_10'], df_joined['EMA_50'])
df_joined['EMA10_EMA100'] = get_diff_percent(df_joined['EMA_10'], df_joined['EMA_100'])
df_joined['EMA10_EMA200'] = get_diff_percent(df_joined['EMA_10'], df_joined['EMA_200'])

df_joined['close_up_EMA20'] = cross(df_joined['close'], df_joined['EMA_20'], dir='up')
df_joined['close_up_EMA50'] = cross(df_joined['close'], df_joined['EMA_50'], dir='up')
df_joined['close_up_EMA100'] = cross(df_joined['close'], df_joined['EMA_100'], dir='up')
df_joined['close_up_EMA200'] = cross(df_joined['close'], df_joined['EMA_200'], dir='up')

df_joined['close_lo_EMA20'] = cross(df_joined['close'], df_joined['EMA_20'], dir='down')
df_joined['close_lo_EMA50'] = cross(df_joined['close'], df_joined['EMA_50'], dir='down')
df_joined['close_lo_EMA100'] = cross(df_joined['close'], df_joined['EMA_100'], dir='down')
df_joined['close_lo_EMA200'] = cross(df_joined['close'], df_joined['EMA_200'], dir='down')

df_joined['EMA10_up_EMA20'] = cross(df_joined['close'], df_joined['EMA_20'], dir='up')
df_joined['EMA10_up_EMA50'] = cross(df_joined['close'], df_joined['EMA_50'], dir='up')
df_joined['EMA10_up_EMA100'] = cross(df_joined['close'], df_joined['EMA_100'], dir='up')
df_joined['EMA10_up_EMA200'] = cross(df_joined['close'], df_joined['EMA_200'], dir='up')

df_joined['EMA10_lo_EMA20'] = cross(df_joined['close'], df_joined['EMA_20'], dir='down')
df_joined['EMA10_lo_EMA50'] = cross(df_joined['close'], df_joined['EMA_50'], dir='down')
df_joined['EMA10_lo_EMA100'] = cross(df_joined['close'], df_joined['EMA_100'], dir='down')
df_joined['EMA10_lo_EMA200'] = cross(df_joined['close'], df_joined['EMA_200'], dir='down')


# RSI
df_joined['up'] = np.clip(df_joined['diff_close'], 0, df_joined['diff_close'])
df_joined['down'] = np.clip(df_joined['diff_close'], df_joined['diff_close'], 0).abs()
df_joined['up_avg'] = df_joined['up'].rolling(window=14).mean()
df_joined['down_avg'] = df_joined['down'].rolling(window=14).mean()
df_joined['RS'] = get_ratio(df_joined['up_avg'], df_joined['down_avg'])
df_joined['RSI'] = 100 - 100 / (df_joined['RS'] + 1)
df_joined['RSI_70'] = compare(df_joined['RSI'], 70, 'greater')
df_joined['RSI_30'] = compare(df_joined['RSI'], 30, 'smaller')


# FSI & SSI
df_joined['FSI'] = (df_joined['close'] - df_joined['close'].shift().rolling(window=14).min()) / (df_joined['close'].shift().rolling(window=14).max() - df_joined['close'].shift().rolling(window=14).min()) * 100
df_joined['SSI'] = df_joined['FSI'].rolling(window=3).mean()
df_joined['FSI_up_SSI'] = cross(df_joined['FSI'], df_joined['SSI'], 'up')
df_joined['FSI_lo_SSI'] = cross(df_joined['FSI'], df_joined['SSI'], 'down')
df_joined['FSI_80'] = compare(df_joined['FSI'], 80, 'greater')
df_joined['FSI_20'] = compare(df_joined['FSI'], 20, 'smaller')


# BOU & BOL
df_joined['BOU'] = df_joined['SMA_10'] + 2 * df_joined['close'].rolling(window=20).std()
df_joined['BOL'] = df_joined['SMA_10'] - 2 * df_joined['close'].rolling(window=20).std()
df_joined['close_BOU'] = df_joined['close'] - df_joined['BOU']
df_joined['close_BOL'] = df_joined['close'] - df_joined['BOL']
df_joined['close_BOU_%'] = get_diff_percent(df_joined['close'], df_joined['BOU'])
df_joined['close_BOL_%'] = get_diff_percent(df_joined['close'], df_joined['BOL'])
df_joined['close_BOU_yn'] = compare(df_joined['close'], df_joined['BOU'], dir='greater')
df_joined['BOL_close_yn'] = compare(df_joined['BOL'], df_joined['close'], dir='greater')

if update:
  df_joined.iloc[201:].to_csv(path+'data_update.csv', index=False)
  df_joined = pd.read_csv(path+'data_update.csv')

df_joined

,date,open,close,high,low,volume1,value1,volume2,value2,buy_order,...,FSI_80,FSI_20,BOU,BOL,close_BOU,close_BOL,close_BOU_%,close_BOL_%,close_BOU_yn,BOL_close_yn
0,2023-10-13,5800.0,5780.0,5800.0,5490.0,113600.0,0.64,0.0,0.0,175.0,...,-1,1,6293.717489,5474.282511,-513.717489,305.717489,-0.081624,0.055846,-1,-1


In [ ]:
df_joined.loc[df_joined.isna().any(axis=1), df_joined.isna().any(axis=0)]

""


#SAVE RESULT

In [ ]:
if update:
  pd.concat([pd.read_csv(path+'data.csv'), df_joined], axis=0).to_csv(path+'data.csv', index=False)
else:
  df_joined.iloc[201:].to_csv(path+'data.csv', index=False)

df_final = pd.read_csv(path+'data.csv')
df_final

,date,open,close,high,low,volume1,value1,volume2,value2,buy_order,...,FSI_80,FSI_20,BOU,BOL,close_BOU,close_BOL,close_BOU_%,close_BOL_%,close_BOU_yn,BOL_close_yn
0,4/20/2011,8500.0,8600.0,8800.0,8500.0,5330.0,45435000.00,0.0,0.0,20.0,...,-1,1,10320.902900,8099.097096,-1720.902904,500.902904,-0.166740,0.061847,-1,-1
1,4/21/2011,9000.0,8800.0,9000.0,8800.0,11040.0,98358000.00,0.0,0.0,19.0,...,-1,1,10274.112220,8005.887781,-1474.112219,794.112220,-0.143478,0.099191,-1,-1
2,4/22/2011,9000.0,8500.0,9000.0,8500.0,1960.0,16670000.00,0.0,0.0,9.0,...,-1,1,10190.697730,7889.302271,-1690.697729,610.697729,-0.165906,0.077408,-1,-1
3,4/25/2011,8800.0,8700.0,8800.0,8500.0,6610.0,56837000.00,0.0,0.0,10.0,...,-1,1,10132.754140,7827.245861,-1432.754139,872.754139,-0.141398,0.111502,-1,-1
4,4/26/2011,8700.0,8500.0,8700.0,8400.0,8210.0,69769000.00,0.0,0.0,11.0,...,-1,1,10034.506330,7745.493671,-1534.506329,754.506329,-0.152923,0.097412,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,2023-10-09,5960.0,5950.0,5960.0,5850.0,5700.0,0.03,0.0,0.0,63.0,...,-1,1,6395.860985,5360.139015,-445.860985,589.860985,-0.069711,0.110046,-1,-1
3115,2023-10-10,5950.0,5910.0,6000.0,5900.0,27600.0,0.16,0.0,0.0,73.0,...,-1,1,6365.597124,5402.402876,-455.597124,507.597124,-0.071572,0.093958,-1,-1
3116,2023-10-11,5990.0,5900.0,5990.0,5750.0,27000.0,0.16,0.0,0.0,96.0,...,-1,1,6343.572095,5434.427905,-443.572095,465.572095,-0.069925,0.085671,-1,-1
3117,2023-10-12,5950.0,5900.0,6000.0,5850.0,28700.0,0.17,0.0,0.0,80.0,...,-1,1,6324.769919,5461.230081,-424.769919,438.769919,-0.067160,0.080343,-1,-1


In [ ]:
df_final.loc[df_final.isna().any(axis=1), df_final.isna().any(axis=0)]

""


In [ ]:
# driver.quit()
# driver2.quit()